# Complete Black-Litterman Optimization

| | | | 
|----------------------------------------------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| n=4 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=4 $returns \sim N(\mu,\sigma)$ Independent  | n=4 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=8 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=8 $returns \sim N(\mu,\sigma)$ Independent  | n=8 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=16 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9 | n=16 $returns \sim N(\mu,\sigma)$ Independent | n=16 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5) |

using the method laid out in 4.2, Black-Litterman portfolio optimization

using https://python-advanced.quantecon.org/black_litterman.html

## Read Packages

In [6]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import random
import trace
import pickle
import sklearn.covariance

In [7]:
import pypfopt as pyp
import warnings
from pypfopt.black_litterman import BlackLittermanModel
warnings.filterwarnings("ignore")


## Read Data

In [8]:
## Read the Data
file_name = "PortfolioOptimizationData.pkl"

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

file_name_os = "PortfolioOptimizationData_Zeros.pkl"

open_file_os = open(file_name_os, "rb")
loaded_list_os = pickle.load(open_file_os)
open_file_os.close()

In [9]:
data_n4_corr = loaded_list[0]
data_n8_corr = loaded_list[1]
data_n16_corr = loaded_list[2]

data_n4_ind = loaded_list[3]
data_n8_ind = loaded_list[4]
data_n16_ind = loaded_list[5]

data_n4_neg = loaded_list[6]
data_n8_neg = loaded_list[7]
data_n16_neg = loaded_list[8]

data_n4_corr_os = loaded_list_os[0]
data_n8_corr_os = loaded_list_os[1]
data_n16_corr_os = loaded_list_os[2]

data_n4_ind_os = loaded_list_os[3]
data_n8_ind_os = loaded_list_os[4]
data_n16_ind_os = loaded_list_os[5]

data_n4_neg_os = loaded_list_os[6]
data_n8_neg_os = loaded_list_os[7]
data_n16_neg_os = loaded_list_os[8]

In [5]:
print(data_n4_corr_os[0].mean(axis=0),data_n8_corr_os[1].mean(axis=0))

[1.24823196 1.25177391 4.92820725 4.94449342] [1.23564355 1.26132808 5.02096074 5.01899361 5.03187764 5.01680219
 5.02930775 4.99411216]


## Define Functions

Function that Calculated equillibrium values 

This step generates the sample

In [10]:
n_assets = 4
return_vec = data_n4_corr_os[0]
np.mean(return_vec, axis = 0)

array([1.24823196, 1.25177391, 4.92820725, 4.94449342])

In [47]:
def bl(n_assets, n_obs, return_vec):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    S = pyp.risk_models.CovarianceShrinkage(return_vec).ledoit_wolf() # pd.DataFrame(return_vec).corr()
    #view = pd.Series(np.mean(return_vec, axis = 0))
    Q = np.reshape(np.mean(return_vec, axis = 0),(n_assets,1))[0:4,:] # 4 views
    P = np.array(
        [
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
        ]
    )
    delta = np.random.normal(1)
    
    bl = pyp.BlackLittermanModel(S, pi = "equal", Q = Q, P = P)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, S)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

In [48]:
def bl_os(n_assets, n_obs, return_vec, return_vec_os):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    S = pyp.risk_models.CovarianceShrinkage(return_vec).ledoit_wolf()
    #view = pd.Series(np.mean(return_vec_os, axis = 0)) #for absolute certainty
    Q = np.reshape(np.mean(return_vec_os, axis = 0),(n_assets,1))[0:4,:] # 4 views
    P = np.array(
        [
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
            np.random.dirichlet(np.ones(n_assets), size=1)[0],
        ]
    )
    delta = np.random.normal(1)
    
    bl = pyp.BlackLittermanModel(S, pi = "equal", Q = Q, P = P)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, S)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

Notes: There's no way around it, I have to give it a view, a vector of data, and absolute views. Otherwise all hell breaks loose.

In [49]:
def one_corr_optimization(n_assets, n_obs, data, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        return_vec = data[i]
        weights, returns, risks = bl(n_assets, n_obs, return_vec) 
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res

In [50]:
def one_corr_optimization_zeros(n_assets, n_obs, data, data_os):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        return_vec = data[i]
        return_vec_os = data_os[i]
        weights, returns, risks = bl_os(n_assets, n_obs, return_vec, return_vec_os)
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res

# Perform Optimization
## Correlated

## $n=4$

In [51]:
n_assets = 4
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [52]:
n4_weights_corr, n4_returns_corr, n4_risks_corr = one_corr_optimization(n_assets, n_obs, data_n4_corr, r) #

In [171]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_corr, axis=0),3)))

weights (mean) = [0.239 0.251 0.257 0.253] 
returns (mean) = [4.029 4.022 4.047 3.953] 
risks (mean) =[1.014 1.014 1.014 1.015]


In [151]:
print('weights (std) = '+str(np.round(np.std(n4_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_corr, axis=0),3)))

weights (std) = [0.096 0.092 0.09  0.089] 
returns (std) = [0.122 0.13  0.136 0.168] 
risks (std) =[2.563 3.403 3.797 2.33 ]


In [53]:
n4_weights_corr_os, n4_returns_corr_os, n4_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, data_n4_corr, data_n4_corr_os)

In [173]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_corr_os, axis=0),3)))

weights (mean) = [0.255 0.275 0.262 0.208] 
returns (mean) = [1.823 1.961 2.794 2.73 ] 
risks (mean) =[1.033 1.034 1.023 1.024]


In [15]:
print('weights (std) = '+str(np.round(np.std(n4_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_corr_os, axis=0),3)))

weights (std) = [0.024 0.026 0.079 0.059] 
returns (std) = [0.074 0.082 0.064 0.048] 
risks (std) =[2.554 3.397 3.806 2.334]


## $ n = 8$

In [54]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [55]:
n8_weights_corr, n8_returns_corr, n8_risks_corr = one_corr_optimization(n_assets, n_obs, data_n8_corr, r) 

In [18]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_corr, axis=0),3)))

weights (mean) = [0.122 0.133 0.118 0.109 0.124 0.12  0.125 0.149] 
returns (mean) = [4.383 4.343 4.394 4.424 4.377 4.396 4.372 4.293] 
risks (mean) =[28.088 27.54  27.768 28.871 27.693 28.629 27.974 26.632]


In [19]:
print('weights (std) = '+str(np.round(np.std(n8_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_corr, axis=0),3)))

weights (std) = [0.032 0.032 0.03  0.028 0.027 0.035 0.028 0.029] 
returns (std) = [0.119 0.112 0.115 0.107 0.091 0.143 0.099 0.099] 
risks (std) =[2.954 3.485 2.222 3.357 2.632 5.761 2.771 2.395]


In [56]:
n8_weights_corr_os, n8_returns_corr_os, n8_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, data_n8_corr, data_n8_corr_os)

In [21]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_corr_os, axis=0),3)))

weights (mean) = [-0.441 -0.442  0.317  0.296  0.324  0.283  0.319  0.343] 
returns (mean) = [3.16  3.124 3.669 3.715 3.657 3.814 3.669 3.643] 
risks (mean) =[28.088 27.54  27.768 28.871 27.693 28.629 27.974 26.632]


In [22]:
print('weights (std) = '+str(np.round(np.std(n8_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_corr_os, axis=0),3)))

weights (std) = [0.027 0.028 0.042 0.042 0.038 0.054 0.041 0.035] 
returns (std) = [0.134 0.118 0.091 0.087 0.08  0.143 0.079 0.075] 
risks (std) =[2.954 3.485 2.222 3.357 2.632 5.761 2.771 2.395]


## $n=16$

In [57]:
n_assets = 16
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [58]:
n16_weights_corr, n16_returns_corr, n16_risks_corr = one_corr_optimization(n_assets, n_obs, data_n16_corr, r) 

In [25]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_corr, axis=0),3)))

weights (mean) = [0.053 0.062 0.065 0.058 0.062 0.07  0.065 0.062 0.06  0.067 0.053 0.062
 0.063 0.069 0.061 0.069] 
returns (mean) = [4.693 4.646 4.638 4.675 4.652 4.615 4.639 4.649 4.662 4.629 4.697 4.652
 4.648 4.615 4.656 4.624] 
risks (mean) =[28.953 28.139 28.107 27.839 28.287 27.281 27.849 28.465 28.441 28.192
 29.063 27.742 27.706 27.949 28.532 27.676]


In [26]:
print('weights (std) = '+str(np.round(np.std(n16_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_corr, axis=0),3)))

weights (std) = [0.026 0.023 0.021 0.024 0.023 0.023 0.024 0.026 0.026 0.024 0.021 0.023
 0.023 0.023 0.027 0.023] 
returns (std) = [0.12  0.121 0.103 0.113 0.117 0.107 0.116 0.133 0.128 0.125 0.126 0.102
 0.11  0.099 0.131 0.108] 
risks (std) =[3.257 4.041 5.387 3.319 4.714 2.888 2.987 3.38  3.829 3.713 7.226 3.045
 3.188 3.831 3.44  4.254]


In [59]:
n16_weights_corr_os, n16_returns_corr_os, n16_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, data_n16_corr, data_n16_corr_os)

In [28]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_corr_os, axis=0),3)))

weights (mean) = [-0.461 -0.471  0.135  0.119  0.135  0.15   0.133  0.111  0.127  0.163
  0.134  0.136  0.147  0.133  0.153  0.156] 
returns (mean) = [3.747 3.726 4.303 4.386 4.297 4.242 4.316 4.415 4.335 4.16  4.289 4.3
 4.247 4.315 4.199 4.212] 
risks (mean) =[28.953 28.139 28.107 27.839 28.287 27.281 27.849 28.465 28.441 28.192
 29.063 27.742 27.706 27.949 28.532 27.676]


In [29]:
print('weights (std) = '+str(np.round(np.std(n16_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_corr_os, axis=0),3)))

weights (std) = [0.028 0.033 0.026 0.029 0.029 0.028 0.03  0.032 0.031 0.032 0.029 0.028
 0.029 0.028 0.034 0.03 ] 
returns (std) = [0.137 0.141 0.105 0.112 0.114 0.099 0.113 0.132 0.123 0.125 0.129 0.099
 0.103 0.097 0.122 0.101] 
risks (std) =[3.257 4.041 5.387 3.319 4.714 2.888 2.987 3.38  3.829 3.713 7.226 3.045
 3.188 3.831 3.44  4.254]


## Independent

In [60]:
n_assets = 4
n_obs = 1000
r = np.eye(n_assets)

In [61]:
n4_weights_ind, n4_returns_ind, n4_risks_ind = one_corr_optimization(n_assets, n_obs, data_n4_ind, r) 

In [32]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_ind, axis=0),3)))

weights (mean) = [0.25  0.251 0.25  0.249] 
returns (mean) = [2.625 2.624 2.62  2.625] 
risks (mean) =[27.569 27.522 27.618 27.654]


In [33]:
print('weights (std) = '+str(np.round(np.std(n4_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_ind, axis=0),3)))

weights (std) = [0.007 0.007 0.007 0.007] 
returns (std) = [0.025 0.028 0.031 0.021] 
risks (std) =[1.44  1.568 1.576 1.677]


In [62]:
n4_weights_ind_os, n4_returns_ind_os, n4_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, data_n4_ind, data_n4_ind_os)

In [35]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_ind_os, axis=0),3)))

weights (mean) = [0.111 0.112 0.389 0.388] 
returns (mean) = [0.748 0.749 2.622 2.625] 
risks (mean) =[27.569 27.522 27.618 27.654]


In [36]:
print('weights (std) = '+str(np.round(np.std(n4_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_ind_os, axis=0),3)))

weights (std) = [0.005 0.005 0.009 0.009] 
returns (std) = [0.017 0.019 0.022 0.019] 
risks (std) =[1.44  1.568 1.576 1.677]


## $n=8$

In [63]:
n_assets = 8
n_obs = 1000
r = np.eye(n_assets)

In [64]:
n8_weights_ind, n8_returns_ind, n8_risks_ind = one_corr_optimization(n_assets, n_obs, data_n8_ind, r) 

In [39]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_ind, axis=0),3)))

weights (mean) = [0.126 0.125 0.126 0.124 0.125 0.125 0.125 0.125] 
returns (mean) = [2.566 2.556 2.561 2.558 2.568 2.567 2.566 2.566] 
risks (mean) =[27.388 27.516 27.467 27.662 27.501 27.447 27.597 27.579]


In [40]:
print('weights (std) = '+str(np.round(np.std(n8_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_ind, axis=0),3)))

weights (std) = [0.004 0.003 0.003 0.003 0.004 0.004 0.004 0.004] 
returns (std) = [0.032 0.032 0.036 0.041 0.042 0.033 0.036 0.034] 
risks (std) =[1.45  1.383 1.344 1.387 1.38  1.552 1.488 1.547]


In [65]:
n8_weights_ind_os, n8_returns_ind_os, n8_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, data_n8_ind, data_n8_ind_os)

In [42]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_ind_os, axis=0),3)))

weights (mean) = [0.041 0.041 0.154 0.152 0.153 0.153 0.153 0.153] 
returns (mean) = [0.69  0.684 2.563 2.558 2.568 2.567 2.566 2.564] 
risks (mean) =[27.388 27.516 27.467 27.662 27.501 27.447 27.597 27.579]


In [43]:
print('weights (std) = '+str(np.round(np.std(n8_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_ind_os, axis=0),3)))

weights (std) = [0.003 0.002 0.004 0.004 0.005 0.005 0.005 0.005] 
returns (std) = [0.03  0.028 0.034 0.035 0.038 0.031 0.033 0.029] 
risks (std) =[1.45  1.383 1.344 1.387 1.38  1.552 1.488 1.547]


## $n=16$

In [66]:
n_assets = 16
n_obs = 1000
r = np.eye(n_assets)

In [67]:
n16_weights_ind, n16_returns_ind, n16_risks_ind = one_corr_optimization(n_assets, n_obs, data_n16_ind, r) 

In [49]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_ind, axis=0),3)))

weights (mean) = [0.063 0.063 0.062 0.063 0.063 0.062 0.062 0.062 0.063 0.062 0.063 0.062
 0.062 0.063 0.062 0.062] 
returns (mean) = [0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062
 0.062 0.062 0.062 0.062] 
risks (mean) =[28.452 28.374 28.483 28.395 28.47  28.598 28.416 28.387 28.431 28.401
 28.337 28.408 28.461 28.304 28.364 28.475]


In [47]:
print('weights (std) = '+str(np.round(np.std(n16_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_ind, axis=0),3)))

weights (std) = [0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.001 0.002 0.002] 
returns (std) = [0.047 0.053 0.044 0.05  0.036 0.048 0.047 0.043 0.043 0.043 0.043 0.048
 0.039 0.04  0.039 0.047] 
risks (std) =[2.246 2.278 2.215 2.244 2.291 2.401 2.355 2.215 2.212 2.227 2.243 2.207
 2.247 2.196 2.282 2.26 ]


In [68]:
n16_weights_ind_os, n16_returns_ind_os, n16_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, data_n16_ind, data_n16_ind_os)

In [50]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_ind_os, axis=0),3)))

weights (mean) = [0.018 0.018 0.069 0.069 0.069 0.069 0.069 0.069 0.069 0.069 0.069 0.069
 0.069 0.069 0.069 0.069] 
returns (mean) = [0.648 0.655 2.539 2.53  2.526 2.525 2.532 2.532 2.533 2.534 2.524 2.532
 2.528 2.531 2.543 2.531] 
risks (mean) =[27.893 27.85  27.909 27.858 27.923 28.065 27.876 27.859 27.9   27.866
 27.777 27.87  27.897 27.773 27.819 27.944]


In [52]:
print('weights (std) = '+str(np.round(np.std(n16_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_ind_os, axis=0),3)))

weights (std) = [0.001 0.001 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.001 0.002 0.002] 
returns (std) = [0.043 0.045 0.042 0.048 0.035 0.044 0.048 0.042 0.043 0.039 0.04  0.047
 0.038 0.038 0.036 0.042] 
risks (std) =[2.246 2.278 2.215 2.244 2.291 2.401 2.355 2.215 2.212 2.227 2.243 2.207
 2.247 2.196 2.282 2.26 ]


## Negatively Correlated

## $n=4$

In [69]:
n_assets = 4
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [70]:
n4_weights_neg, n4_returns_neg, n4_risks_neg = one_corr_optimization(n_assets, n_obs, data_n4_neg, r) 

In [55]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_neg, axis=0),3)))

weights (mean) = [0.244 0.251 0.253 0.252] 
returns (mean) = [2.502 2.502 2.509 2.486] 
risks (mean) =[79.513 77.851 77.113 77.946]


In [56]:
print('weights (std) = '+str(np.round(np.std(n4_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_neg, axis=0),3)))

weights (std) = [0.019 0.015 0.014 0.015] 
returns (std) = [0.109 0.11  0.102 0.121] 
risks (std) =[129.595 129.801 129.898 129.807]


In [71]:
n4_weights_neg_os, n4_returns_neg_os, n4_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, data_n4_neg, data_n4_neg_os)

In [58]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_neg_os, axis=0),3)))

weights (mean) = [0.118 0.122 0.382 0.379] 
returns (mean) = [0.656 0.653 2.569 2.561] 
risks (mean) =[79.513 77.851 77.113 77.946]


In [59]:
print('weights (std) = '+str(np.round(np.std(n4_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_neg_os, axis=0),3)))

weights (std) = [0.011 0.012 0.02  0.022] 
returns (std) = [0.084 0.085 0.055 0.061] 
risks (std) =[129.595 129.801 129.898 129.807]


## $n=8$

In [72]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [73]:
n8_weights_neg, n8_returns_neg, n8_risks_neg = one_corr_optimization(n_assets, n_obs, data_n8_neg, r) 

In [62]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_neg, axis=0),3)))

weights (mean) = [0.123 0.123 0.126 0.124 0.126 0.127 0.127 0.124] 
returns (mean) = [2.614 2.616 2.609 2.62  2.616 2.601 2.605 2.615] 
risks (mean) =[11788.8   11780.73  11777.977 11778.722 11778.409 11776.92  11777.152
 11820.304]


In [63]:
print('weights (std) = '+str(np.round(np.std(n8_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_neg, axis=0),3)))

weights (std) = [0.014 0.009 0.007 0.009 0.005 0.007 0.005 0.013] 
returns (std) = [0.12  0.128 0.106 0.135 0.123 0.094 0.112 0.12 ] 
risks (std) =[94350.789 94351.755 94352.094 94352.    94352.061 94352.225 94352.202
 94348.467]


In [74]:
n8_weights_neg_os, n8_returns_neg_os, n8_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, data_n8_neg, data_n8_neg_os)

In [65]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_neg_os, axis=0),3)))

weights (mean) = [0.038 0.038 0.154 0.152 0.155 0.156 0.155 0.152] 
returns (mean) = [0.733 0.736 2.601 2.604 2.604 2.594 2.595 2.606] 
risks (mean) =[11788.8   11780.73  11777.977 11778.722 11778.409 11776.92  11777.152
 11820.304]


In [66]:
print('weights (std) = '+str(np.round(np.std(n8_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_neg_os, axis=0),3)))

weights (std) = [0.008 0.007 0.008 0.01  0.007 0.009 0.006 0.015] 
returns (std) = [0.108 0.112 0.087 0.097 0.095 0.079 0.09  0.1  ] 
risks (std) =[94350.789 94351.755 94352.094 94352.    94352.061 94352.225 94352.202
 94348.467]


## $n=16$

In [75]:
n_assets = 16
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [76]:
n16_weights_neg, n16_returns_neg, n16_risks_neg = one_corr_optimization(n_assets, n_obs, data_n16_neg, r) 

In [69]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_neg, axis=0),3)))

weights (mean) = [0.062 0.062 0.063 0.063 0.062 0.062 0.062 0.063 0.063 0.063 0.063 0.062
 0.063 0.063 0.062 0.062] 
returns (mean) = [2.573 2.584 2.57  2.57  2.57  2.571 2.569 2.57  2.577 2.572 2.564 2.579
 2.575 2.57  2.567 2.575] 
risks (mean) =[10950.891 10933.715 10931.137 10932.633 10966.737 10992.492 11048.607
 10933.507 10931.159 10933.829 10938.266 10940.265 10932.237 10934.607
 10946.648 10951.126]


In [70]:
print('weights (std) = '+str(np.round(np.std(n16_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_neg, axis=0),3)))

weights (std) = [0.006 0.004 0.004 0.004 0.008 0.007 0.006 0.002 0.003 0.003 0.005 0.006
 0.002 0.005 0.006 0.006] 
returns (std) = [0.146 0.159 0.13  0.129 0.128 0.159 0.143 0.132 0.151 0.142 0.12  0.156
 0.136 0.131 0.132 0.144] 
risks (std) =[35600.646 35605.269 35606.038 35605.613 35596.642 35593.679 35591.529
 35605.366 35606.039 35605.344 35603.986 35603.391 35605.726 35605.021
 35601.676 35601.096]


In [77]:
n16_weights_neg_os, n16_returns_neg_os, n16_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, data_n16_neg, data_n16_neg_os)

In [72]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_neg_os, axis=0),3)))

weights (mean) = [0.017 0.017 0.07  0.07  0.068 0.068 0.068 0.069 0.07  0.07  0.069 0.068
 0.07  0.069 0.069 0.069] 
returns (mean) = [0.697 0.706 2.565 2.566 2.565 2.567 2.565 2.566 2.572 2.566 2.561 2.574
 2.57  2.565 2.562 2.569] 
risks (mean) =[10950.891 10933.715 10931.137 10932.633 10966.737 10992.492 11048.607
 10933.507 10931.159 10933.829 10938.266 10940.265 10932.237 10934.607
 10946.648 10951.126]


In [73]:
print('weights (std) = '+str(np.round(np.std(n16_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_neg_os, axis=0),3)))

weights (std) = [0.004 0.004 0.004 0.005 0.008 0.007 0.007 0.003 0.003 0.003 0.006 0.006
 0.003 0.006 0.007 0.006] 
returns (std) = [0.139 0.155 0.115 0.12  0.111 0.143 0.122 0.118 0.133 0.122 0.111 0.142
 0.123 0.114 0.115 0.125] 
risks (std) =[35600.646 35605.269 35606.038 35605.613 35596.642 35593.679 35591.529
 35605.366 35606.039 35605.344 35603.986 35603.391 35605.726 35605.021
 35601.676 35601.096]


## Save the Results

In [78]:
BlackLittermanResults_Weights = [n4_weights_corr, n8_weights_corr, n16_weights_corr, n4_weights_corr_os, 
                              n8_weights_corr_os, n16_weights_corr_os, n4_weights_ind, n8_weights_ind,
                              n16_weights_ind, n4_weights_ind_os, n8_weights_ind_os, n16_weights_ind_os,
                              n4_weights_neg, n8_weights_neg, n16_weights_neg, n4_weights_neg_os, 
                              n8_weights_neg_os, n16_weights_neg_os]

file_name = "BlackLittermanResultsWeights.pkl"
open_file = open(file_name, "wb")
pickle.dump(BlackLittermanResults_Weights, open_file)
open_file.close()

In [148]:
BlackLittermanResults_Returns = [n4_returns_corr, n8_returns_corr, n16_returns_corr, n4_returns_corr_os, 
                              n8_returns_corr_os, n16_returns_corr_os, n4_returns_ind, n8_returns_ind,
                              n16_returns_ind, n4_returns_ind_os, n8_returns_ind_os, n16_returns_ind_os,
                              n4_returns_neg, n8_returns_neg, n16_returns_neg, n4_returns_neg_os, 
                              n8_returns_neg_os, n16_returns_neg_os]

file_name = "BlackLittermanResultsReturns.pkl"
open_file = open(file_name, "wb")
pickle.dump(BlackLittermanResults_Returns, open_file)
open_file.close()

In [149]:
BlackLittermanResults_Risks = [n4_risks_corr, n8_risks_corr, n16_risks_corr, n4_risks_corr_os, 
                              n8_risks_corr_os, n16_risks_corr_os, n4_risks_ind, n8_risks_ind,
                              n16_risks_ind, n4_risks_ind_os, n8_risks_ind_os, n16_risks_ind_os,
                              n4_risks_neg, n8_risks_neg, n16_risks_neg, n4_risks_neg_os, 
                              n8_risks_neg_os, n16_risks_neg_os]

file_name = "BlackLittermanResultsRisks.pkl"
open_file = open(file_name, "wb")
pickle.dump(BlackLittermanResults_Risks, open_file)
open_file.close()

## Play with the view's values

I let the optimization run with 100 repetitions. In each repetition I randomly select an amount between -10% and 10% of the default views. Then I select the maximum returns and found that returns are maximized when views are 9.9% higher than the prior value.

So, The higher the views, the higher the calculated returns. 
So... I'll just use the views equal to the prior so that views don't sway the results. right.?

In [1]:
def bl_views(n_assets, n_obs, return_vec, views):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    market_prices = pd.Series(np.random.randn(n_assets)) 
    rng = np.random.default_rng()
    cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
    market_prior = pd.Series(np.mean(return_vec + 5, axis = 0))
    view = pd.Series(views)#np.ones(n_assets)*(1/n_assets)+ 5.0)
    
    bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

In [118]:
def views_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_diff = np.zeros((99,n_assets))
    return_diff = np.zeros((99,n_assets))
    risks_diff = np.zeros((99,n_assets))
    view_diff = np.zeros((99,n_assets))
    
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    views = np.zeros((99,n_assets))
    
    rng = np.random.default_rng()
    return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    
    for i in range(99):
        # Make default calculation
        default = np.mean(return_vec + 5, axis = 0)
        default_weights, default_ret, default_rsk = bl_views(n_assets, n_obs, return_vec, default)
        default_weights = list(default_weights.items())
        wt = [x[1] for x in default_weights]
        weight_def = wt
        return_def = np.array(default_ret).T
        risks_def = np.diagonal(np.array(default_rsk))
        
        views[i] = np.mean(return_vec + 5, axis = 0) + np.round(np.random.uniform(-0.1,0.1,1),3) * np.mean(return_vec + 5, axis=0)
        weights, returns, risks = bl_views(n_assets, n_obs, return_vec, views[i]) 
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
        view_diff[i] = (views[i]-default)/default * 100
        weight_diff[i,:] = (weight_res[i] - weight_def)/weight_def * 100
        return_diff[i,:] = (return_res[i] - return_def)/return_def * 100
        risks_diff[i,:] = (risks_res[i] - risks_def)/risks_def * 100
    return views, view_diff, weight_diff, return_res, risks_diff

In [28]:
n_assets = 4
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [119]:
views, views_diff, views_weights, views_returns, views_risks = views_optimization(n_assets, n_obs, r) 

In [95]:
out = pd.DataFrame(np.hstack((views_diff, views_returns)))

In [120]:
views_diff[views_returns.sum(axis=1).argmax()]
#np.where(x == np.max(views_returns, axis=0))
#views_returns.sum(axis=1)#.argmax()

array([9.9, 9.9, 9.9, 9.9])

In [105]:
type(views_returns[1,1])

numpy.float64

In [113]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(out)

       0     1     2     3         4         5         6         7
0    3.8   3.8   3.8   3.8  5.052776  5.044410  5.079386  5.068058
1   -1.5  -1.5  -1.5  -1.5  4.921375  4.913226  4.947292  4.936259
2   -9.4  -9.4  -9.4  -9.4  4.725995  4.717771  4.750990  4.739886
3    2.7   2.7   2.7   2.7  5.025504  5.017183  5.051971  5.040703
4   11.8  11.8  11.8  11.8  5.251559  5.244740  5.279746  5.267524
5   14.6  14.6  14.6  14.6  5.320539  5.311729  5.348559  5.336630
6   18.1  18.1  18.1  18.1  5.407314  5.398360  5.435791  5.423667
7   18.9  18.9  18.9  18.9  5.427148  5.418162  5.455729  5.443562
8    3.9   3.9   3.9   3.9  5.055256  5.046885  5.081879  5.070545
9    5.2   5.2   5.2   5.2  5.087486  5.079062  5.114279  5.102873
10  15.7  15.7  15.7  15.7  5.347777  5.339086  5.376023  5.363993
11  17.5  17.5  17.5  17.5  5.392438  5.383509  5.420837  5.408747
12  10.6  10.6  10.6  10.6  5.221368  5.212722  5.248865  5.237159
13  15.1  15.1  15.1  15.1  5.332926  5.324098  5.361025  5.34

Archived:
    def bl(n_assets, n_obs, return_vec):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    market_prices = pd.Series(np.random.randn(n_assets)) # This needs to be Rf - Rm1
    cov_struct = return_vec
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
    data = pd.Series(np.mean(return_vec + 5, axis = 0))
    view = pd.Series(np.mean(return_vec + 5, axis = 0))#np.ones(n_assets)*(1/n_assets)+ 5.0)
    
    bl = pyp.BlackLittermanModel(S, pi=data, absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl
    
    ### This is a test - do not run
#r = np.random.uniform(0.8, 0.9, (4,4))
#rng = np.random.default_rng()
#return_vec = rng.multivariate_normal(np.zeros(4), cov = r, size = 100)


#market_prices = pd.Series(np.mean(return_vec)) 
#rng = np.random.default_rng()
#cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
#S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
#delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
#market_prior = pd.Series(np.random.randn(n_assets))
#view = pd.Series(np.ones(n_assets)*(1/n_assets))
    
#bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
#bl_return = bl.bl_returns()

#ef = pyp.EfficientFrontier(bl_return, r)
#bl.bl_weights(delta)
#weights = bl.clean_weights()

#S_bl = bl.bl_cov()
#print(cov_struct, S, S_bl)
#np.mean(return_vec, axis = 0)

In [ ]:
def one_corr_optimization_zeros(n_assets, n_obs, data, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        return_vec = data[i]
        _75_perct = int(len(return_vec[:,0])*3/4)
        return_vec[random.sample(list(range(1000)), _75_perct),0] = 0
        np.random.seed(i*5)
        return_vec[random.sample(list(range(1000)), _75_perct),1] = 0
        
        weights, returns, risks = bl(n_assets, n_obs, return_vec)
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res